In this file, high resolution patches are predicted by the neural network for the validation years.  

In [1]:
!pip install xarray
!pip install netCDF4
!pip install h5netcdf
!pip install rasterio

In [2]:
import google.cloud.storage
import io
import numpy as np
import matplotlib.pyplot as plt
import xarray
import rasterio
from natsort import natsorted 
import tempfile
from rasterio.io import MemoryFile
from rasterio.plot import show
import tensorflow as tf
import pickle
from keras import backend as K

Using TensorFlow backend.


In [3]:
client = google.cloud.storage.Client()

In [4]:
bucket = client.get_bucket('era-ml-upressing')

In [5]:
#select period
selection = np.s_[1460:]  #select 1980, 1980-01-02 to 1980-12-31 leap year

#ERA-20C
files_folder_era20C= !gsutil ls -r 'gs://era-ml-upressing/Predict'
files_era20C_list = natsorted(files_folder_era20C[2:])
files_era20C= [file[22:] for file in files_era20C_list[selection]]

#ERA5
files_folder_era5= !gsutil ls -r 'gs://era-ml-upressing/Test'
files_era5_list = natsorted(files_folder_era5[2:])
files_era5= [file[22:] for file in files_era5_list[:]]


In [6]:
files_era20C

['Predict/20000101-120km.tif',
 'Predict/20000102-120km.tif',
 'Predict/20000103-120km.tif',
 'Predict/20000104-120km.tif',
 'Predict/20000105-120km.tif',
 'Predict/20000106-120km.tif',
 'Predict/20000107-120km.tif',
 'Predict/20000108-120km.tif',
 'Predict/20000109-120km.tif',
 'Predict/20000110-120km.tif',
 'Predict/20000111-120km.tif',
 'Predict/20000112-120km.tif',
 'Predict/20000113-120km.tif',
 'Predict/20000114-120km.tif',
 'Predict/20000115-120km.tif',
 'Predict/20000116-120km.tif',
 'Predict/20000117-120km.tif',
 'Predict/20000118-120km.tif',
 'Predict/20000119-120km.tif',
 'Predict/20000120-120km.tif',
 'Predict/20000121-120km.tif',
 'Predict/20000122-120km.tif',
 'Predict/20000123-120km.tif',
 'Predict/20000124-120km.tif',
 'Predict/20000125-120km.tif',
 'Predict/20000126-120km.tif',
 'Predict/20000127-120km.tif',
 'Predict/20000128-120km.tif',
 'Predict/20000129-120km.tif',
 'Predict/20000130-120km.tif',
 'Predict/20000131-120km.tif',
 'Predict/20000201-120km.tif',
 'Predic

In [7]:
#when model is trained on custom loss function, this function needs to be specified here to load the model
def loss_function(y_true, y_pred):
    return (K.mean(K.square(K.cast(K.greater(y_true, 0.7), tf.float32) * (y_pred - y_true)*3))) + (K.mean(K.square(K.cast(K.less_equal(y_true, 0.7), tf.float32) * (y_pred - y_true))))

In [8]:
# define model used for prediction, compile=False when you used a custom loss function
model = tf.keras.models.load_model('gs://era-ml-upressing/saved_models/autoencoder29', compile=False)

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss=loss_function)

In [10]:
#Here all prediction patches are downscaled with the neural network
import tqdm
from rasterio.transform import Affine
from dateutil.parser import parse
#open file with tempfile rasterio:

dates = []
data_array = None
predxlist = None

#blob = bucket.get_blob(blobname)
for file in tqdm.tqdm(files_era20C):
    blob = bucket.get_blob(file)                                 #get file from folder
    stream = io.BytesIO(blob.download_as_string())               #download file 
    date = parse(file[8:12] + '-' + file[12:14] + '-' + file[14:16])
    
    with MemoryFile(stream) as memfile:                          
        with memfile.open() as dataset:                          #open file, dataset contains all data
            ds = xarray.open_rasterio(dataset)
            transform = dataset.transform  
            if predxlist is None:
                predxlist = ds.expand_dims(time=[date], axis=0)
                
            else:
                predxlist_new = ds.expand_dims(time=[date], axis=0)
                predxlist = xarray.concat([predxlist, predxlist_new], dim='time')     
            a = np.ndarray.flatten(ds[0].data)
            data = a**(1/5)     
            b = np.where(data< 1e-4, 0 , data)
            #rescale data, first ^0.4 and then the scaler function
            c = b.reshape(-1,1)
            scaler = pickle.load(open('scaler_new_1.pkl', 'rb'))
            d = scaler.transform(c)
            predx_norm = d.reshape(1, 64, 64, 1)
            predy_norm = model.predict(predx_norm)               #predict high resolution image from ERA20C
            #e = predy_norm.numpy()
            f = np.ndarray.flatten(predy_norm)
            g = f.reshape(-1,1)                                  #scale data back to original value range
            h = scaler.inverse_transform(g)
            i = h ** 5
            Z = i.reshape(256,256)
            newstream = io.BytesIO()    
            #create geotiff file with precipitation data and coordinates
            with rasterio.open(
                newstream, 'w', driver='GTiff', 
                height=Z.shape[0], width=Z.shape[1], 
                count=1, dtype=Z.dtype, crs='+proj=latlong', 
                transform=transform* Affine.scale(0.25)) as raster:
                raster.write(Z, 1)
            newstream.seek(0)
            with MemoryFile(newstream) as memfile2:                          
                with memfile2.open(count=1, 
                                   driver='GTiff',
                                   height=Z.shape[0], width=Z.shape[1],
                                   dtype=Z.dtype, crs='+proj=latlong', 
                                   transform=transform* Affine.scale(0.25)) as dataset2: 
                    if data_array is None:
                        # this should only happen once
                        # create an initial dataset on the first run
                        data_array = xarray.open_rasterio(dataset2)
                        data_array = data_array.expand_dims(time=[date], axis=0)
                    else:
                        # otherwise append it to the dataset
                        data_array_new = xarray.open_rasterio(dataset2)
                        data_array_new = data_array_new.expand_dims(time=[date], axis=0)
                        data_array = xarray.concat([data_array, data_array_new], dim='time')                    

100%|██████████| 1097/1097 [08:00<00:00,  2.28it/s]


In [11]:
data_array

<xarray.DataArray (time: 1097, band: 1, y: 256, x: 256)>
array([[[[5.38239055e-06, 7.74633918e-06, 9.27437759e-06, ...,
          6.06523099e-06, 4.86000363e-06, 4.65951371e-06],
         [5.95529764e-06, 6.12349413e-06, 6.28010002e-06, ...,
          3.72015984e-06, 3.26795907e-06, 4.16599869e-06],
         [7.05841512e-06, 6.75484580e-06, 6.98789427e-06, ...,
          4.92707295e-06, 3.31062392e-06, 2.55311761e-06],
         ...,
         [1.37894662e-04, 1.37907424e-04, 1.29708889e-04, ...,
          9.53648851e-05, 7.44043064e-05, 8.55072940e-05],
         [1.91238229e-04, 2.08046622e-04, 1.65498219e-04, ...,
          1.26898594e-04, 9.69688408e-05, 6.46159897e-05],
         [1.91103463e-04, 1.65446167e-04, 1.47399624e-04, ...,
          1.16543299e-04, 1.06348911e-04, 7.23185003e-05]]],


       [[[1.43360859e-03, 1.21017173e-03, 1.27254229e-03, ...,
          4.09747910e-04, 4.43154102e-04, 4.01503377e-04],
         [1.58674712e-03, 1.17451255e-03, 1.27845863e-03, ...,
          3.53641342e-04, 4.20936092e-04, 4.68041486e-04],
         [1.43310451e-03, 1.30438968e-03, 1.05432852e-03, ...,
...
         [4.00864956e-05, 2.01780567e-05, 2.55049454e-05, ...,
          4.29543434e-06, 7.07730396e-06, 9.23853440e-06],
         [3.16540718e-05, 1.98700891e-05, 2.45366427e-05, ...,
          3.26860936e-06, 5.14338171e-06, 2.42100782e-06]]],


       [[[1.76586182e-05, 2.51589336e-05, 2.26737739e-05, ...,
          1.27706990e-05, 1.31695306e-05, 1.15773692e-05],
         [1.71979409e-05, 2.70069359e-05, 3.17675112e-05, ...,
          8.40423672e-06, 1.04403816e-05, 7.89101614e-06],
         [2.97485676e-05, 3.39064572e-05, 3.73658622e-05, ...,
          8.58424846e-06, 8.37409789e-06, 5.01148361e-06],
         ...,
         [6.23928063e-05, 4.83260519e-05, 3.92111433e-05, ...,
          5.86088689e-04, 3.85990686e-04, 9.29788454e-04],
         [6.41505030e-05, 5.37498381e-05, 4.60101583e-05, ...,
          5.91741409e-04, 6.03736087e-04, 7.75079592e-04],
         [5.66251729e-05, 3.63238141e-05, 4.34359354e-05, ...,
          3.55455501e-04, 5.28028235e-04, 6.67978020e-04]]]],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2003-01-01
  * band     (band) int64 1
  * y        (y) float64 82.82 82.55 82.28 82.01 ... 14.91 14.64 14.37 14.1
  * x        (x) float64 -28.37 -28.1 -27.83 -27.56 ... 39.54 39.81 40.08 40.35
Attributes:
    transform:      (0.26949458523585645, 0.0, -28.505053926976956, 0.0, -0.2...
    crs:            +proj=longlat +ellps=WGS84 +no_defs=True
    res:            (0.26949458523585645, 0.26949458523585645)
    is_tiled:       0
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area

In [12]:
#scaling factor is defined to account for the precipitation loss (see section scaling in discussion)
sumx = predxlist.sum(dim=('x','y'))
sumpred = data_array.sum(dim=('x','y'))
factor= (sumx*16)/sumpred
data_array1 = data_array*factor

In [60]:
#check
(data_array1/data_array)[4]

<xarray.DataArray (band: 1, y: 256, x: 256)>
array([[[1.5305439, 1.5305439, 1.5305438, ..., 1.5305439, 1.5305439,
         1.5305438],
        [1.5305439, 1.530544 , 1.5305439, ..., 1.5305439, 1.5305439,
         1.5305439],
        [1.5305439, 1.5305439, 1.5305438, ..., 1.5305439, 1.5305439,
         1.5305438],
        ...,
        [1.530544 , 1.5305439, 1.530544 , ..., 1.5305439, 1.5305439,
         1.5305439],
        [1.5305439, 1.5305439, 1.5305439, ..., 1.5305439, 1.5305439,
         1.5305439],
        [1.5305439, 1.5305439, 1.5305439, ..., 1.5305439, 1.5305439,
         1.5305439]]], dtype=float32)
Coordinates:
    time     datetime64[ns] 1980-01-06
  * band     (band) int64 1
  * y        (y) float64 82.82 82.55 82.28 82.01 ... 14.91 14.64 14.37 14.1
  * x        (x) float64 -28.37 -28.1 -27.83 -27.56 ... 39.54 39.81 40.08 40.35

In [18]:
# squeeze out band dimension and add metadata to data array
da = data_array1.squeeze()
da.y.attrs['units'] = 'degrees_north'
da.y.attrs['standard_name'] = 'latitude'
da.y.attrs['long_name'] = 'latitude'
da.x.attrs['units'] = 'degrees_east'
da.x.attrs['standard_name'] = 'longitude'
da.x.attrs['long_name'] = 'longitude'
da.time.attrs['standard_name'] = 'time'
da.time.attrs['long_name'] = 'time'
da = da[:731, ...]
da = da.to_dataset(name='precipitation')
da.attrs['Conventions'] = 'CF-1.8'
da.precipitation.attrs['units'] = 'm'
da = da.drop('band')
da = da.rename({'x': 'longitude', 'y': 'latitude'})


In [19]:
da

<xarray.Dataset>
Dimensions:        (latitude: 256, longitude: 256, time: 731)
Coordinates:
  * time           (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2001-12-31
  * latitude       (latitude) float64 82.82 82.55 82.28 ... 14.64 14.37 14.1
  * longitude      (longitude) float64 -28.37 -28.1 -27.83 ... 39.81 40.08 40.35
Data variables:
    precipitation  (time, latitude, longitude) float32 6.534646e-05 ... 1.420...
Attributes:
    Conventions:  CF-1.8

In [22]:
#convert data array to netcdf file
da.to_netcdf('DownscaledERA20C-2000-2001_new.nc')
                    

In [23]:
#upload netcdf file to bucket
!gsutil cp 'DownscaledERA20C-2000-2001_new.nc' gs://era-ml-upressing/wflow_files

Copying file://DownscaledERA20C-2000-2001_new.nc [Content-Type=application/x-netcdf]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][182.8 MiB/182.8 MiB]                                                
Operation completed over 1 objects/182.8 MiB.                                    
